### ASPEKTU NOSKAŅOJUMA NOTEIKŠANA

In [2]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
import xml.etree.ElementTree as ET
import pandas as pd
from openai import OpenAI

def parse_xml_sentiment_pred(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                aspects.append(target)
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects
          })
    return pd.DataFrame(data)

df_test = parse_xml_sentiment_pred('new1.xml')
print(df_test)

# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="" # for API key
)
def predict_sentiment(aspect, sentence):
    prompt = (
        "Your task is to do aspect-based sentiment analysis.\n"
        "Given an aspect term and a sentence, predict the sentiment towards the given aspect (positive, negative, neutral).\n\n"
        f"Aspect term: '{aspect}'\nSentence: \"{sentence}\"\n\n"
        "Respond ONLY in this exact format: aspect:polarity\n"
        "Example: apkalpošana:positive\n"
        "Do not provide any additional text or explanation.\n"
    )
    completion = client.chat.completions.create(
        extra_body={},
        model="mistralai/mistral-small-3.1-24b-instruct:free",
        messages=[{"role": "user", "content": prompt}]
    )
    pred = completion.choices[0].message.content.strip().lower()
# OpenAI. ChatGPT o3-mini-high. Uzvedne: uzlabo manu kodu tā, lai nevis tiktu aprēķinātas precision, recall un f1 vērtības, bet iegūtie rezultāti tiktu ierakstīti failā, kurš pēc tam tiks konstanti papildināts ar citiem rezultātiem un iepriekš ierakstītie rezultāti netiks dzēsti ārā. iegūtajiem rezultātiem ir jābūt ierakstītiem tā, ka, ja teikumā ir viens aspekts, tad vienā rindiņā ir tikai aspect:polarity, bet ja ir 2 un vairāk aspektu, tad vienā rindiņā jābūt aspect1:polarity1, aspect2:polarity2. tiem ir jābūt tādā secībā, kādā ir teikumi un kādā esot aspekti pašā teikumā https://chatgpt.com/ [izmantots 2025-04-02]
    if ':' in pred:
        return pred
    else:
        return f"{aspect}:{pred}"

output_file = 'results_sentiments.txt'
with open(output_file, "a", encoding="utf-8") as f:
  for idx, row in df_test.iterrows():
    sentence_text = row['text']
    aspect_preds = []
    if len(row['aspects']) == 0:
      continue
    for aspect in row['aspects']:
      if aspect.lower() == "null":
        continue
      pred = predict_sentiment(aspect, sentence_text)
      print(pred)
      aspect_preds.append(pred.strip())
    if aspect_preds:
      f.write(", ".join(aspect_preds) + "\n")
print(f"\n Results have been saved to {output_file}")


                     id                                               text  \
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.   
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...   
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...   
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...   
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.   
5  lv_OPaps_000000002:1  Vienīgi cenas pēdējā laikā diezgan palielināju...   
6  lv_OPaps_000000002:2  Latvijas alus ir labā cenā, bet izlejamais imp...   
7  lv_OPaps_000000002:3  Cepelīni garšīgi, teikšu, ka pagaidām labākie,...   
8   lv_OPaps_00000002:4  Baraviku zupa garšīga, bet, manuprāt, 11€ ir m...   
9  lv_OPaps_000000002:5         Ķiploku grauzdiņi ir ļoti gardi un viegli.   

                                    aspects  
0                                   [vieta]  
1                   [ēdieniem, alus izlase]  
2  

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target and target.lower() != "null":
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects,
              'sentiments': sentiments
          })
    return pd.DataFrame(data)

df_test = parse_xml('new1.xml')
print(df_test.head())

# OpenAI. GPT-4o. Uzvedne: nepieciešams pārveidot šo kodu tā, lai tiktu vēlreiz izgūti dati no XML faila, lai varētu salīdzināt pareizos rezultātus ar lielā valodas modeļa atrastajiem rezultātiem, kuri ir saglabāti atsevišķā .txt failā aspect:polarity formā, un lai tad varētu aprēķināt metriku rezultātus. Rezultāti skaitās pareizi tikai tad, ja iegūtie rezultāti pilnībā sakrīt ar īstajiem rezultātiem gan kvalitatīvā ziņā, gan kvantitatīvā ziņā https://chatgpt.com/ [izmantots 2025-04-02]
def load_predictions(file_path):
    predictions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            aspect_polarity_pairs = line.split(',')
            aspects = []
            sentiments = []
            for pair in aspect_polarity_pairs:
                if ':' in pair:
                    parts = pair.split(':', maxsplit = 1)
                    if len(parts) != 2:
                      continue
                    aspect,polarity = parts
                    aspects.append(aspect.strip())
                    sentiments.append(polarity.strip())
            predictions.append({
                'aspects': aspects,
                'sentiments': sentiments
            })
    return predictions

def evaluate(true_df, pred_list):
    y_true = []
    y_pred = []
    mismatches = []

    for i, row in true_df.iterrows():
        true_aspects = row['aspects']
        true_sentiments = row['sentiments']

        pred_aspects = pred_list[i]['aspects']
        pred_sentiments = pred_list[i]['sentiments']

        if true_aspects == pred_aspects and true_sentiments == pred_sentiments:
            y_true.append(1)
            y_pred.append(1)
        else:
            y_true.append(1)
            y_pred.append(0)
            mismatches.append({
                'text': row['text'],
                'true_aspects': true_aspects,
                'true_sentiments': true_sentiments,
                'pred_aspects': pred_aspects,
                'pred_sentiments': pred_sentiments
            })
    print(f"\nAccuracy: {accuracy_score(y_true, y_pred)}")
    print(f"\nPrecision: {precision_score(y_true, y_pred)}")
    print(f"\nRecall: {recall_score(y_true, y_pred)}")
    print(f"\nF1 score: {f1_score(y_true, y_pred)}")
    if mismatches:
        for m in mismatches:
            print(f"\nTeikums: {m['text']}")
            print(f"PAREIZS: {list(zip(m['true_aspects'], m['true_sentiments']))}")
            print(f"MODEĻA: {list(zip(m['pred_aspects'], m['pred_sentiments']))}")

xml_path = 'new1.xml'
predictions_path = 'results_sentiments.txt'
df_test = parse_xml(xml_path)
predictions = load_predictions(predictions_path)
assert len(df_test) == len(predictions), "Teikumu skaits nesakrīt ar modeļa rezultātu skaitu!"
evaluate(df_test, predictions)

                     id                                               text  \
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.   
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...   
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...   
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...   
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.   

                   aspects            sentiments  
0                  [vieta]            [positive]  
1  [ēdieniem, alus izlase]  [positive, positive]  
2           [lielopa gaļa]            [positive]  
3            [apkalpošana]            [positive]  
4       [ēdieni, dzērieni]  [positive, positive]  

Accuracy: 0.7777777777777778

Precision: 1.0

Recall: 0.7777777777777778

F1 score: 0.875

Teikums: Latvijas alus ir labā cenā, bet izlejamais importa alus jau maksā ap 7€ par puslitru.
PAREIZS: [('latvijas alus', 'positive'

### ASPEKTU IZGŪŠANA

In [5]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
import xml.etree.ElementTree as ET
import pandas as pd
from openai import OpenAI

def parse_xml_aspect_extraction(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        data.append({
            'id': sentence.get('id'),
            'text': text
        })
    return pd.DataFrame(data)

df_test = parse_xml_aspect_extraction('new1.xml')
print(df_test.head())

# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="" # for API key
)
def extract_aspects(sentence):
    prompt = (
        "Your task is to do aspect term extraction.\n"
        "Given a sentence, extract all explicit aspect terms mentioned in the sentence.\n"
        "If there is no explicit aspect in a whole sentence or in an exact part of a sentence, give answer 'null'.\n\n"
        f"Sentence: \"{sentence}\"\n\n"
        "Respond ONLY with aspect terms, comma-separated.\n"
        "Example: apkalpošana, ēdienu, null\n"
        "Do not provide any additional text or explanation and do not guess implicit aspects.\n"
    )
    completion = client.chat.completions.create(
        extra_body={},
        model="mistralai/mistral-small-3.1-24b-instruct:free",
        messages=[{"role": "user", "content": prompt}]
    )
    pred = completion.choices[0].message.content.strip().lower()
    return pred

# OpenAI. GPT-4o. Uzvedne: kā uzlabot šo koda daļu, lai tas būtu piemērots izgūtu aspektu saglabāšanai? https://chatgpt.com/ [izmantots 2025-04-02]
output_file = 'results_aspects.txt'
with open(output_file, "a", encoding="utf-8") as f:
    for idx, row in df_test.iterrows():
        sentence_text = row['text']
        pred_aspects = extract_aspects(sentence_text)
        if pred_aspects:
            f.write(pred_aspects + "\n")
print(f"\n Results have been saved to {output_file}")

                     id                                               text
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.

 Results have been saved to results_aspects.txt


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                if target:
                  aspects.append(target.lower())
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects
          })
    return pd.DataFrame(data)

df_test = parse_xml('new1.xml')
print(df_test.head())

# OpenAI. GPT-4o. Uzvedne: nepieciešams pārveidot šo kodu tā, lai tiktu vēlreiz izgūti dati no XML faila, lai varētu salīdzināt pareizos rezultātus ar lielā valodas modeļa atrastajiem rezultātiem, kuri ir saglabāti atsevišķā .txt failā aspect, aspect formā, un lai tad varētu aprēķināt metriku rezultātus. Rezultāti skaitās pareizi tikai tad, ja iegūtie rezultāti pilnībā sakrīt ar īstajiem rezultātiem gan kvalitatīvā ziņā, gan kvantitatīvā ziņā https://chatgpt.com/ [izmantots 2025-04-02]
def load_predictions(file_path):
    predictions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            aspect_list = [aspect.strip().lower() for aspect in line.split(',') if aspect.strip()]
            predictions.append({
                'aspects': aspect_list
            })
    return predictions

def evaluate(true_df, pred_list):
    y_true = []
    y_pred = []
    mismatches = []

    for i, row in true_df.iterrows():
        true_aspects = row['aspects']
        pred_aspects = pred_list[i]['aspects']
        if true_aspects == pred_aspects:
            y_true.append(1)
            y_pred.append(1)
        else:
            y_true.append(1)
            y_pred.append(0)
            mismatches.append({
                'text': row['text'],
                'true_aspects': true_aspects,
                'pred_aspects': pred_aspects
            })
    print(f"\nAccuracy: {accuracy_score(y_true, y_pred)}")
    print(f"\nPrecision: {precision_score(y_true, y_pred)}")
    print(f"\nRecall: {recall_score(y_true, y_pred)}")
    print(f"\nF1 score: {f1_score(y_true, y_pred)}")
    if mismatches:
        for m in mismatches:
            print(f"\nTeikums: {m['text']}")
            print(f"PAREIZS: {list(zip(m['true_aspects']))}")
            print(f"MODEĻA: {list(zip(m['pred_aspects']))}")

xml_path = 'new1.xml'
predictions_path = 'results_aspects.txt'
df_test = parse_xml(xml_path)
predictions = load_predictions(predictions_path)
assert len(df_test) == len(predictions), "Teikumu skaits nesakrīt ar modeļa rezultātu skaitu!"
evaluate(df_test, predictions)

                     id                                               text  \
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.   
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...   
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...   
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...   
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.   

                   aspects  
0                  [vieta]  
1  [ēdieniem, alus izlase]  
2           [lielopa gaļa]  
3            [apkalpošana]  
4       [ēdieni, dzērieni]  

Accuracy: 0.1

Precision: 1.0

Recall: 0.1

F1 score: 0.18181818181818182

Teikums: Lieliska vieta vakariņām.
PAREIZS: [('vieta',)]
MODEĻA: [('vaketa',), ('vakariņām',)]

Teikums: Ļoti draudzīgas cenas ēdieniem un laba vietējā alus izlase.
PAREIZS: [('ēdieniem',), ('alus izlase',)]
MODEĻA: [('draudzīgas cenas ēdieniem',), ('laba vietējā alus izlase',)]



### VIENLAICĪGA ASPEKTU IZGŪŠANA UN TO NOSKAŅOJUMU NOTEIKŠANA

In [7]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
import xml.etree.ElementTree as ET
import pandas as pd
from openai import OpenAI

def parse_xml_full_absa(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        data.append({
            'id': sentence.get('id'),
            'text': text
        })
    return pd.DataFrame(data)

df_test = parse_xml_full_absa('new1.xml')
print(df_test.head())

# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="" # for API key
)
def extract_aspects_predict_sentiment(sentence):
    prompt = (
        "Your task is to do aspect-based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms mentioned in the sentence and predict the sentiments towards the extracted aspects (positive, negative, neutral).\n\n"
        "If there is no explicit aspect in a whole sentence or in an exact part of a sentence, give answer 'null'.\n\n"
        f"Sentence: \"{sentence}\"\n\n"
        "Respond ONLY in this exact format: aspect1:polarity1, aspect2:polarity2\n"
        "Example: apkalpošana:positive, null:negative\n"
        "Do not provide any additional text or explanation and do not guess implicit aspects.\n"
    )
    completion = client.chat.completions.create(
        extra_body={},
        model="google/gemma-3-27b-it:free",
        messages=[{"role": "user", "content": prompt}]
    )
    pred = completion.choices[0].message.content.strip().lower()

# OpenAI. ChatGPT o3-mini-high. Uzvedne: uzlabo manu kodu tā, lai nevis tiktu aprēķinātas precision, recall un f1 vērtības, bet iegūtie rezultāti tiktu ierakstīti failā, kurš pēc tam tiks konstanti papildināts ar citiem rezultātiem un iepriekš ierakstītie rezultāti netiks dzēsti ārā. iegūtajiem rezultātiem ir jābūt ierakstītiem tā, ka, ja teikumā ir viens aspekts, tad vienā rindiņā ir tikai aspect:polarity, bet ja ir 2 un vairāk aspektu, tad vienā rindiņā jābūt aspect1:polarity1, aspect2:polarity2. tiem ir jābūt tādā secībā, kādā ir teikumi un kādā esot aspekti pašā teikumā https://chatgpt.com/ [izmantots 2025-04-02]
    if ':' in pred:
        return pred
    else:
        return f"{aspect}:{pred}"

output_file = 'results_full.txt'
with open(output_file, "a", encoding="utf-8") as f:
    for idx, row in df_test.iterrows():
        sentence_text = row['text']
        pred = extract_aspects_predict_sentiment(sentence_text)
        if pred:
            f.write(pred + "\n")
print(f"\n Results have been saved to {output_file}")

                     id                                               text
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.

 Results have been saved to results_full.txt


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target:
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects,
              'sentiments': sentiments
          })
    return pd.DataFrame(data)

df_test = parse_xml('new1.xml')
print(df_test.head())

# OpenAI. GPT-4o. Uzvedne: nepieciešams pārveidot šo kodu tā, lai tiktu vēlreiz izgūti dati no XML faila, lai varētu salīdzināt pareizos rezultātus ar lielā valodas modeļa atrastajiem rezultātiem, kuri ir saglabāti atsevišķā .txt failā aspect:polarity formā, un lai tad varētu aprēķināt metriku rezultātus. Rezultāti skaitās pareizi tikai tad, ja iegūtie rezultāti pilnībā sakrīt ar īstajiem rezultātiem gan kvalitatīvā ziņā, gan kvantitatīvā ziņā https://chatgpt.com/ [izmantots 2025-04-02]
def load_predictions(file_path):
    predictions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            aspect_polarity_pairs = line.split(',')
            aspects = []
            sentiments = []
            for pair in aspect_polarity_pairs:
                if ':' in pair:
                    parts = pair.split(':', maxsplit = 1)
                    if len(parts) != 2:
                      continue
                    aspect,polarity = parts
                    aspects.append(aspect.strip())
                    sentiments.append(polarity.strip())
            predictions.append({
                'aspects': aspects,
                'sentiments': sentiments
            })
    return predictions

def evaluate(true_df, pred_list):
    y_true = []
    y_pred = []
    mismatches = []

    for i, row in true_df.iterrows():
        true_aspects = row['aspects']
        true_sentiments = row['sentiments']

        pred_aspects = pred_list[i]['aspects']
        pred_sentiments = pred_list[i]['sentiments']

        if true_aspects == pred_aspects and true_sentiments == pred_sentiments:
            y_true.append(1)
            y_pred.append(1)
        else:
            y_true.append(1)
            y_pred.append(0)
            mismatches.append({
                'text': row['text'],
                'true_aspects': true_aspects,
                'true_sentiments': true_sentiments,
                'pred_aspects': pred_aspects,
                'pred_sentiments': pred_sentiments
            })
    print(f"\nAccuracy: {accuracy_score(y_true, y_pred)}")
    print(f"\nPrecision: {precision_score(y_true, y_pred)}")
    print(f"\nRecall: {recall_score(y_true, y_pred)}")
    print(f"\nF1 score: {f1_score(y_true, y_pred)}")
    if mismatches:
        for m in mismatches:
            print(f"\nTeikums: {m['text']}")
            print(f"PAREIZS: {list(zip(m['true_aspects'], m['true_sentiments']))}")
            print(f"MODEĻA: {list(zip(m['pred_aspects'], m['pred_sentiments']))}")

xml_path = 'new1.xml'
predictions_path = 'results_full.txt'
df_test = parse_xml(xml_path)
predictions = load_predictions(predictions_path)
assert len(df_test) == len(predictions), "Teikumu skaits nesakrīt ar modeļa rezultātu skaitu!"
evaluate(df_test, predictions)

                     id                                               text  \
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.   
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...   
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...   
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...   
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.   

                   aspects            sentiments  
0                  [vieta]            [positive]  
1  [ēdieniem, alus izlase]  [positive, positive]  
2           [lielopa gaļa]            [positive]  
3            [apkalpošana]            [positive]  
4       [ēdieni, dzērieni]  [positive, positive]  

Accuracy: 0.3

Precision: 1.0

Recall: 0.3

F1 score: 0.46153846153846156

Teikums: Ļoti draudzīgas cenas ēdieniem un laba vietējā alus izlase.
PAREIZS: [('ēdieniem', 'positive'), ('alus izlase', 'positive')]
MODEĻA: [('cena

### VIENLAICĪGA ASPEKTU IZGŪŠANA UN TO NOSKAŅOJUMU NOTEIKŠANA AR PIEMĒRU

In [9]:
# OpenRouter. Sample code and API for DeepSeek V3 0324 (free). Tiešsaiste. OpenRouter. Pieejams: https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free/api. [skatīts 2025-03-29].
import xml.etree.ElementTree as ET
import pandas as pd
from openai import OpenAI

def parse_xml_full_absa_with_examples(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        data.append({
            'id': sentence.get('id'),
            'text': text
        })
    return pd.DataFrame(data)

df_test = parse_xml_full_absa_with_examples('new1.xml')
print(df_test.head())

# OpenAI. ChatGPT o3-mini-high. Uzvedne: kā man uzlabot šo kodu, lai tas izmantotu no XML izgūtos teikumus, aspektus un noskaņojumus noskaņojuma noteikšanai aspektiem un vēl novērtētu modeļa veiktspēju? https://chatgpt.com/ [izmantots 2025-03-29]
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="" # for API key
)
def extract_aspects_predict_sentiment(sentence):
    prompt = (
        "Your task is to do aspect-based sentiment analysis.\n"
        "Given a sentence, extract all explicit aspect terms mentioned in the sentence and predict the sentiments towards the aspects (positive, negative, neutral).\n"
        "If there is no explicit aspect in a whole sentence or in an exact part of a sentence, give answer 'null'.\n\n"
        "Example:\n"
        "Sentence: Vīnu saraksts ir interesants, un tajā ir daudz labu piedāvājumu.\n"
        "Vīnu saraksts:positive, Vīnu saraksts:positive\n\n"
        f"Sentence: \"{sentence}\"\n\n"
        "Respond ONLY in this exact format: aspect1:polarity1, aspect2:polarity2\n"
        "Example: apkalpošana:positive, null:negative\n"
        "Do not provide any additional text or explanation and do not guess implicit aspects.\n"
    )
    completion = client.chat.completions.create(
        extra_body={},
        model="google/gemma-3-27b-it:free",
        messages=[{"role": "user", "content": prompt}]
    )
    pred = completion.choices[0].message.content.strip().lower()

# OpenAI. ChatGPT o3-mini-high. Uzvedne: uzlabo manu kodu tā, lai nevis tiktu aprēķinātas precision, recall un f1 vērtības, bet iegūtie rezultāti tiktu ierakstīti failā, kurš pēc tam tiks konstanti papildināts ar citiem rezultātiem un iepriekš ierakstītie rezultāti netiks dzēsti ārā. iegūtajiem rezultātiem ir jābūt ierakstītiem tā, ka, ja teikumā ir viens aspekts, tad vienā rindiņā ir tikai aspect:polarity, bet ja ir 2 un vairāk aspektu, tad vienā rindiņā jābūt aspect1:polarity1, aspect2:polarity2. tiem ir jābūt tādā secībā, kādā ir teikumi un kādā esot aspekti pašā teikumā https://chatgpt.com/ [izmantots 2025-04-02]
    if ':' in pred:
        return pred
    else:
        return f"{aspect}:{pred}"

output_file = 'results_full_with_examples.txt'
with open(output_file, "a", encoding="utf-8") as f:
    for idx, row in df_test.iterrows():
        sentence_text = row['text']
        pred = extract_aspects_predict_sentiment(sentence_text)
        if pred:
            f.write(pred + "\n")
print(f"\n Results have been saved to {output_file}")

                     id                                               text
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.

 Results have been saved to results_full_with_examples.txt


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for sentence in root.iter('sentence'):
        text = sentence.find('text').text.strip()
        opinions = sentence.find('Opinions')
        aspects = []
        sentiments = []
        if opinions is not None:
            for opinion in opinions.findall('Opinion'):
                target = opinion.get('target')
                polarity = opinion.get('polarity')
                if target:
                  aspects.append(target.lower())
                  sentiments.append(polarity)
        if aspects:
          data.append({
              'id': sentence.get('id'),
              'text': text,
              'aspects': aspects,
              'sentiments': sentiments
          })
    return pd.DataFrame(data)

df_test = parse_xml('new1.xml')
print(df_test.head())

# OpenAI. GPT-4o. Uzvedne: nepieciešams pārveidot šo kodu tā, lai tiktu vēlreiz izgūti dati no XML faila, lai varētu salīdzināt pareizos rezultātus ar lielā valodas modeļa atrastajiem rezultātiem, kuri ir saglabāti atsevišķā .txt failā aspect:polarity formā, un lai tad varētu aprēķināt metriku rezultātus. Rezultāti skaitās pareizi tikai tad, ja iegūtie rezultāti pilnībā sakrīt ar īstajiem rezultātiem gan kvalitatīvā ziņā, gan kvantitatīvā ziņā https://chatgpt.com/ [izmantots 2025-04-02]
def load_predictions(file_path):
    predictions = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            aspect_polarity_pairs = line.split(',')
            aspects = []
            sentiments = []
            for pair in aspect_polarity_pairs:
                if ':' in pair:
                    parts = pair.split(':', maxsplit = 1)
                    if len(parts) != 2:
                      continue
                    aspect,polarity = parts
                    aspects.append(aspect.strip())
                    sentiments.append(polarity.strip())
            predictions.append({
                'aspects': aspects,
                'sentiments': sentiments
            })
    return predictions

def evaluate(true_df, pred_list):
    y_true = []
    y_pred = []
    mismatches = []

    for i, row in true_df.iterrows():
        true_aspects = row['aspects']
        true_sentiments = row['sentiments']

        pred_aspects = pred_list[i]['aspects']
        pred_sentiments = pred_list[i]['sentiments']

        if true_aspects == pred_aspects and true_sentiments == pred_sentiments:
            y_true.append(1)
            y_pred.append(1)
        else:
            y_true.append(1)
            y_pred.append(0)
            mismatches.append({
                'text': row['text'],
                'true_aspects': true_aspects,
                'true_sentiments': true_sentiments,
                'pred_aspects': pred_aspects,
                'pred_sentiments': pred_sentiments
            })
    print(f"\nAccuracy: {accuracy_score(y_true, y_pred)}")
    print(f"\nPrecision: {precision_score(y_true, y_pred)}")
    print(f"\nRecall: {recall_score(y_true, y_pred)}")
    print(f"\nF1 score: {f1_score(y_true, y_pred)}")
    if mismatches:
        for m in mismatches:
            print(f"\nTeikums: {m['text']}")
            print(f"PAREIZS: {list(zip(m['true_aspects'], m['true_sentiments']))}")
            print(f"MODEĻA: {list(zip(m['pred_aspects'], m['pred_sentiments']))}")

xml_path = 'new1.xml'
predictions_path = 'results_full_with_examples.txt'
df_test = parse_xml(xml_path)
predictions = load_predictions(predictions_path)
assert len(df_test) == len(predictions), "Teikumu skaits nesakrīt ar modeļa rezultātu skaitu!"
evaluate(df_test, predictions)

                     id                                               text  \
0  lv_OPaps_000000001:0                          Lieliska vieta vakariņām.   
1  lv_OPaps_000000001:1  Ļoti draudzīgas cenas ēdieniem un laba vietējā...   
2  lv_OPaps_000000001:2  Ēdieniem īpaši garda un augstvērtīga lielopa g...   
3  lv_OPaps_000000001:3  Perfekta apkalpošana, prot ieteikt lieliskus ē...   
4  lv_OPaps_000000002:0                   Ļoti garšīgi ēdieni un dzērieni.   

                   aspects            sentiments  
0                  [vieta]            [positive]  
1  [ēdieniem, alus izlase]  [positive, positive]  
2           [lielopa gaļa]            [positive]  
3            [apkalpošana]            [positive]  
4       [ēdieni, dzērieni]  [positive, positive]  

Accuracy: 0.4

Precision: 1.0

Recall: 0.4

F1 score: 0.5714285714285714

Teikums: Lieliska vieta vakariņām.
PAREIZS: [('vieta', 'positive')]
MODEĻA: [('lieliska vieta', 'positive')]

Teikums: Perfekta apkalpošana, prot iete